# Online Monitoring 

## 2 Basic Steps of Online Monitoring  

* real-time data
  * setup Tag in  DataBase
  * sampling and send data to DataBase

* online analysis
  * setup out Tag in  DataBase
  * Get real-time data from Database and analyze
  * send results to Database


##  1 Real-time Data 

/bootProcessInformation/db/pyredis.py

### 1.1 setup Tag in DataBase


In [17]:
# -*- coding: utf-8 -*-
import redis

conn = redis.Redis('localhost')

def TagDefToRedisHashKey(tagdeflist):
    pipe = conn.pipeline()
    for element in tagdeflist:
        pipe.hmset(element['id'], 
                   {'desc': element['desc'], 'value': "-10000", 'ts': ""})
    pipe.execute()

def TagDefFromRedisHash(tagdeflist):
    taglist = []
    for element in tagdeflist:
        htag = conn.hgetall(element['id'])
        taglist.append(htag)
    return taglist


* real-time data
  * setup Tag in DataBase

In [18]:
unittagdeflist=[]
unittagdeflist.append({'id':"CSDC.DCS2AI.2JZA2226",'desc':"二号机组高压缸入口压力"})
unittagdeflist.append({'id':"CSDC.DCS2AI.2JZA2408",'desc':"二号机组高压缸入口温度"})
unittagdeflist.append({'id':"CSDC.DCS2AI.2JZA2230",'desc':"二号机组高压缸出口压力"})
unittagdeflist.append({'id':"CSDC.DCS2AI.2JZA2459",'desc':"二号机组高压缸出口温度"})
unittagdeflist.append({'id':"CSDC.DCS2AI.2JZA2264",'desc':"大气压力"})

TagDefToRedisHashKey(unittagdeflist)

taglist=TagDefFromRedisHash(unittagdeflist)
print(taglist[0])

{b'desc': b'\xe4\xba\x8c\xe5\x8f\xb7\xe6\x9c\xba\xe7\xbb\x84\xe9\xab\x98\xe5\x8e\x8b\xe7\xbc\xb8\xe5\x85\xa5\xe5\x8f\xa3\xe5\x8e\x8b\xe5\x8a\x9b', b'value': b'-10000', b'ts': b''}


* real-time data
  * sampling and send data to DataBase

In [19]:
# -*- coding: utf-8 -*-
from datetime import *

def SendToRedisHash(tagvaluelist):
    pipe = conn.pipeline()

    curtime = datetime.now()

    for element in tagvaluelist:
        pipe.hmset(element['id'], 
                   {'value': element['value'], 'ts': curtime})
    pipe.execute()


In [20]:
unittagvaluelist=[]
unittagvaluelist.append({'id':"CSDC.DCS2AI.2JZA2226",'value':16.38525})
unittagvaluelist.append({'id':"CSDC.DCS2AI.2JZA2408",'value':538.37})
unittagvaluelist.append({'id':"CSDC.DCS2AI.2JZA2230",'value':3.035039})
unittagvaluelist.append({'id':"CSDC.DCS2AI.2JZA2459",'value':313.2931})
unittagvaluelist.append({'id':"CSDC.DCS2AI.2JZA2264",'value':100.5281})

SendToRedisHash(unittagvaluelist)

print(unittagvaluelist[0])
print(conn.hmget(unittagvaluelist[0]['id'], 'value', 'ts'))
 

{'value': 16.38525, 'id': 'CSDC.DCS2AI.2JZA2226'}
[b'16.38525', b'2016-05-06 14:26:53.037757']


## 2 online analysis

### 2.1 Get Real-time Data from Redis

In [21]:
# -*- coding: utf-8 -*-
def tagvalue_redis(taglist):
        #        for i in range(self.tagcount):
        #            self.taglist[i]['value']= conn.hget(self.taglist[i]['name'],'value').decode()
        tagcount=len(taglist)
        pipe = conn.pipeline()
        
        for i in range(tagcount):
            pipe.hmget(taglist[i]['id'], 'value', 'ts')
        
        tagvaluelist = pipe.execute()

        for i in range(tagcount):
            taglist[i]['value'] = tagvaluelist[i][0].decode()
            taglist[i]['ts'] = tagvaluelist[i][1].decode()


In [22]:
unittaglist=[]
unittaglist.append({'id':"CSDC.DCS2AI.2JZA2226"})
unittaglist.append({'id':"CSDC.DCS2AI.2JZA2408"})
unittaglist.append({'id':"CSDC.DCS2AI.2JZA2230"})
unittaglist.append({'id':"CSDC.DCS2AI.2JZA2459"})
unittaglist.append({'id':"CSDC.DCS2AI.2JZA2264"})

tagvalue_redis(unittaglist)

print(unittaglist[0])
        

{'value': '16.38525', 'ts': '2016-05-06 14:26:53.037757', 'id': 'CSDC.DCS2AI.2JZA2226'}


### 2.2  Online moitoring 

* set outag
* get real-time data and analyze
* send results 

In [23]:
# -*- coding: utf-8 -*-
from seuif97 import *

def CylinderEff(cylinder):
    """simple function  for cylinde using 'dict' """

    cylinder['inlet']['h'] = pt2h(cylinder['inlet']['p'], cylinder['inlet']['t'])
    cylinder['inlet']['s'] = pt2s(cylinder['inlet']['p'], cylinder['inlet']['t'])

    cylinder['outlet']['h'] = pt2h(cylinder['outlet']['p'], cylinder['outlet']['t'])
    cylinder['outlet']['s'] = pt2s(cylinder['outlet']['p'], cylinder['outlet']['t'])

    # h2s is the specific enthalpy at state 2 for the isentropic turbine
    h2s = ps2h(cylinder['outlet']['p'], cylinder['inlet']['s'])
    
    cylinder['h2s'] = h2s
     
    hds = cylinder['inlet']['h'] - h2s  # isentropic specific enthalpy drop
    hd = cylinder['inlet']['h'] - cylinder['outlet']['h']  # specific enthalpy drop

    cylinder['eff'] = 100 * hd / hds

    return cylinder


In [25]:
unit_outtaglist=[]
unit_outtaglist.append({'id':"DEMO.DCS2AO.HP_EFF_IS",'desc':"二号机组高压缸效率"})

# 1 set out tag
TagDefToRedisHashKey(unit_outtaglist)

# 2 get real-time data
tagvalue_redis(unittaglist)
pam=float(unittaglist[4]['value'])/1000

# 3 analysis

hp = {'inlet':{}, 'outlet':{}, 'h2s':None,'eff':None}

minlet = {'p':None, 't': None, 'h': None, 's':None}
moutlet = {'p': None, 't': None, 'h': None, 's': None}

minlet['p']=float(unittaglist[0]['value'])+pam
minlet['t']=float(unittaglist[1]['value'])
moutlet['p']=float(unittaglist[2]['value'])+pam
moutlet['t']=float(unittaglist[3]['value'])

hp['inlet'] = dict(minlet)
hp['outlet'] = dict(moutlet)

hp = CylinderEff(hp)

print('The internal efficiency =', hp['eff'], '%')

# 4 send result to database--------------------------
unit_outtaglist[0]['value']=hp['eff']
SendToRedisHash(unit_outtaglist)

taglist=TagDefFromRedisHash(unit_outtaglist)
print(taglist[0])

The internal efficiency = 81.71506348039361 %
{b'ts': b'2016-05-06 14:32:22.871045', b'value': b'81.71506348039361', b'desc': b'\xe4\xba\x8c\xe5\x8f\xb7\xe6\x9c\xba\xe7\xbb\x84\xe9\xab\x98\xe5\x8e\x8b\xe7\xbc\xb8\xe6\x95\x88\xe7\x8e\x87'}


## Reference

<ol>
<li>The Redis Documentation. http://redis.io/documentation
<li>JOSIAH L. CARLSON. Redis in Action. Manning Publications Co. 2013 
</ol>